In [17]:
! pip install pandas scikit-learn jieba

In [18]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import jieba

In [21]:
# 定義主目錄
main_folder = '/home/b10173209/test.file/SS/class10(KFold)'

# 分類的清單
categories = ["anger","fear","joy","love","sadness","surprise"]


def custom_tokenizer(text):
    tokens = jieba.lcut(text)
    return tokens



# 建立空的 DataFrame 用於存放結果
results_df = pd.DataFrame(columns=['Category', 'Split', 'Fold', 'Val Accuracy', 'Val Precision', 'Val Recall', 'Val F1 Score',
                                   'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1 Score',
                                   'Val Macro Precision', 'Val Macro Recall', 'Val Macro F1',
                                   'Macro Precision', 'Macro Recall', 'Macro F1', 'C', 'kernel'])

# 定義參數網格
param_grid = {'C': [0.1, 1, 10, 100],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}


In [22]:
# 迴圈跑 kernel、C
for kernel in param_grid['kernel']:
    for C_value in param_grid['C']:
        # 建立空的 DataFrame 用於存放結果
        category_results = []

        print(f'''
##################################################################################
##     Start Running kernel : {kernel} 、 C_values : {C_value}
################################################################################## '''
        )

        # 遍歷每個類別
        for category in categories:
            category_folder = os.path.join(main_folder, category)

            # 遍歷每個子資料夾
            for i in range(5):
                subfolder = f'{category}_{i}'
                subfolder_path = os.path.join(category_folder, subfolder)

                # Debugging: Print folder path
                print(f'資料夾路徑: {subfolder_path}')

                # 檢查資料夾是否存在
                if not os.path.exists(subfolder_path):
                    print(f'資料夾不存在: {subfolder_path}')
                    continue

                # 讀取訓練集
                train_file = f'train_{category}_{i}.csv'
                train_df = pd.read_csv(os.path.join(subfolder_path, train_file))

                # 讀取測試集
                test_file = f'test_{category}_{i}.csv'
                test_df = pd.read_csv(os.path.join(subfolder_path, test_file))

                # 訓練集的標籤轉換為數字
                train_labels = train_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()
                test_labels = test_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                # 使用 StratifiedKFold 進行分層交叉驗證
                num_folds = 5
                skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

                # 遍歷每一折
                for fold, (train_index, val_index) in enumerate(skf.split(train_df, train_labels)):
                    print(f"折數: {fold + 1}/{num_folds}")

                    # 分割訓練和驗證集
                    train_data, val_data = train_df.iloc[train_index], train_df.iloc[val_index]

                    # 訓練集的標籤轉換為數字
                    train_labels_fold = train_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                    # 驗證集的標籤轉換為數字
                    val_labels_fold = val_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                    # 建立 TfidfVectorizer 將文本數據轉換為 TF-IDF 特徵
                    vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=None)
                    X_train_fold = vectorizer.fit_transform(train_data['text'])
                    X_val_fold = vectorizer.transform(val_data['text'])
                    X_test = vectorizer.transform(test_df['text'])

                    svm_model = SVC(kernel=kernel,
                                    C=C_value,
                                    class_weight='balanced',
                                    random_state=42)

                    # 訓練模型
                    svm_model.fit(X_train_fold, train_labels_fold)

                    # 在驗證集上進行預測
                    predictions_val = svm_model.predict(X_val_fold)

                    # 評估模型性能（在驗證集上）
                    accuracy_val = accuracy_score(val_labels_fold, predictions_val)
                    precision_val = precision_score(val_labels_fold, predictions_val, average='weighted')
                    recall_val = recall_score(val_labels_fold, predictions_val, average='weighted')
                    f1_val = f1_score(val_labels_fold, predictions_val, average='weighted')
                    print(f'Validation Accuracy: {accuracy_val}, Precision: {precision_val}, Recall: {recall_val}, F1 Score: {f1_val}')

                    # 在測試集上進行預測
                    predictions_test = svm_model.predict(X_test)

                    # 評估模型性能（在測試集上）
                    accuracy_test = accuracy_score(test_labels, predictions_test)
                    precision_test = precision_score(test_labels, predictions_test, average='weighted')
                    recall_test = recall_score(test_labels, predictions_test, average='weighted')
                    f1_test = f1_score(test_labels, predictions_test, average='weighted')

                    # 計算 Validation Set 上的 Macro Precision、Macro Recall 和 Macro F1
                    macro_precision_val = precision_score(val_labels_fold, predictions_val, average='macro')
                    macro_recall_val = recall_score(val_labels_fold, predictions_val, average='macro')
                    macro_f1_val = f1_score(val_labels_fold, predictions_val, average='macro')

                    # 計算整體（Macro）的指標
                    macro_precision_test = precision_score(test_labels, predictions_test, average='macro')
                    macro_recall_test = recall_score(test_labels, predictions_test, average='macro')
                    macro_f1_test = f1_score(test_labels, predictions_test, average='macro')

                    category_results.append({
                        'Category': category,
                        'Split': f'{category}_{i}',
                        'Fold': fold + 1,
                        'Val Accuracy': accuracy_val,
                        'Val Precision': precision_val,
                        'Val Recall': recall_val,
                        'Val F1 Score': f1_val,
                        'Test Accuracy': accuracy_test,
                        'Test Precision': precision_test,
                        'Test Recall': recall_test,
                        'Test F1 Score': f1_test,
                        'Val Macro Precision': macro_precision_val,
                        'Val Macro Recall': macro_recall_val,
                        'Val Macro F1': macro_f1_val,
                        'Macro Precision': macro_precision_test,
                        'Macro Recall': macro_recall_test,
                        'Macro F1': macro_f1_test,
                        'C': C_value,
                        'kernel': kernel
                    })

        # 將所有結果合併成一個 DataFrame
        results_df = pd.concat([results_df, pd.DataFrame(category_results)])

# 將結果 DataFrame 存入 Excel 檔案
results_excel_path = '/home/b10173209/test.file/SS/all_results.xlsx'
results_df.to_excel(results_excel_path, index=False)


##################################################################################
##     Start Running kernel : linear 、 C_values : 0.1
################################################################################## 
資料夾路徑: /home/b10173209/test.file/SS/class10(KFold)/anger/anger_0
折數: 1/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Validation Accuracy: 0.9434375, Precision: 0.9414831312764455, Recall: 0.9434375, F1 Score: 0.9420526908462539
折數: 2/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Validation Accuracy: 0.944375, Precision: 0.9424446639062535, Recall: 0.944375, F1 Score: 0.9429821322509743
折數: 3/5


...

Validation Accuracy: 0.9534375, Precision: 0.9676571416874156, Recall: 0.9534375, F1 Score: 0.9590877074589863
折數: 4/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Validation Accuracy: 0.936875, Precision: 0.9631639451129475, Recall: 0.936875, F1 Score: 0.9473880099309185
折數: 5/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Validation Accuracy: 0.95125, Precision: 0.964652936650628, Recall: 0.95125, F1 Score: 0.9567734589996938
